# Nichtlineare Optimierung mit Nebenbedingungen

## Dr. Anselm Hudde - 24.01.2022

![](optimization.png)
**Quelle: Venter, G. (2010). Review of optimization techniques.**

In [ ]:
%run nonlinear_programming.py

## Worum geht es bei der nichtlinearen Optimierung?

Bei der Optimierung geht es darum, das Minimum (bzw. das Maximum) einer Funktion $f$ zu finden.

- Wie maximiert ein Unternehmen den Ertrag, unter Berücksichtigung der vorhandenen Ressourcen?

- Wie erreicht man mit einer Portfolioallokation eine maximale Rendite, wenn die erlaubte Volatilität beschränkt ist?

- Wie kann man die richtigen Modellparameter schätzen (statistische Modelle, neuronale Netze)?

## Beispiel aus der Portfoliooptimierung:

Ein Investor will $10\,000$ Euro investieren, wobei diese auf zwei Aktien, $A$ und $B$, aufgeteilt werden.

- Erwartete jährliche Renditen: 

    Aktie $A$: $r_A = 7\%$

    Aktie $B$: $r_{A} = 9\%$
    
    
- Volatilitäten:

    Aktie $A$: $\sigma_A = 20\%$

    Aktie $B$: $\sigma_{B} = 30\%$
    
    
- Korrelation:

    $\rho_{A, B} = 0.7$

Der Investor möchte sein Geld so anlegen, dass er eine erwartete Rendite von $5\%$ erreicht, und dabei so wenig Volatilität wie möglich zulassen.
Die Volatilität eines Portfolios bestehend aus $x_1 \times 10\,000$ € in Aktie $A$ und $x_2 \times 10\,000$ € in Aktie $B$ berechnet sich als

\begin{equation}
f (x_{1}, x_{2})
=
\sqrt{ \sigma_{A}^2 x_1^2 + \sigma_{B}^2 x_2^2 + 2 \sigma_{A} x_1 \sigma_{B} x_2 \rho_{A, B}}.
\end{equation}

Unser Ziel ist also: 

\begin{equation}
\begin{split}
\text{Minimiere }& f(x) & \textit{ (die Volatilität)}\\
\text{wobei }& x_1 + x_2 - 1 \leq 0 & \textit{ (nicht mehr als 100% investieren)}\\
\text{und }& r_{A} x_{1} + r_{B} x_{2} - 0.05 = 0 ~& \textit{ (5% Rendite)}
\end{split}
\end{equation}
gilt.
Wenn wir

\begin{equation}
g(x_1, x_2) = x_1 + x_2 - 1
\end{equation}
und
\begin{equation}
h(x_1, x_2) = r_{A} x_{1} + r_{B} x_{2} - 0.05
\end{equation}

definieren, können wir unser Problem als Beispiel einer allgemeinen Formulierung eines Optimierungsproblems auffassen.

## Formulierung eines nichtlinearen Optimierungsproblems mit Nebenbedingungen

Sei $D \subseteq \mathbb{R}^n$ eine Teilmenge, und seien

\begin{equation}
\begin{split}
f \colon D \to \mathbb{R} \\
g \colon D \to \mathbb{R} \\
h \colon D \to \mathbb{R}
\end{split}
\end{equation}

drei (möglicherweise nichtlineare) Funktionen.

**Ziel der nichtlinearen Optimierung:**

\begin{equation}
\begin{split}
\text{Minimiere }& f(x) \\
\text{wobei }& g(x) \leq 0 \\
\text{und }& h(x) = 0 \text{ gilt.}
\end{split}
\end{equation}

## Beispielfunktion

Wir betrachten die Funktion

\begin{equation}f(x, y) = \sin(x) + 0.05 x^2 + 0.1 y^2, \end{equation}

die wir minimieren wollen:

In [ ]:
import numpy as np

def f(x):
    return np.sin(x[0]) + 0.05*x[0]**2 + 0.1*x[1]**2

surface_plot = plot()
surface_plot.plot_surface(-5,1,-3,3,f)
surface_plot.show()

contour_plot = plot()
contour_plot.plot_contour(-5,1,-3,3,f)
contour_plot.show()

## Das Gradientenverfahren: Idee

Die Idee ist es, immer in die Richtung zu gehen, in die es am steilsten bergauf- bzw. bergab geht.

![](optimization.png)

Aber wie finden wir diese Richtung? Dafür müssen wir die Steigung messen können. 

Doch wie messen wir die Steigung?
Hier kommt der **Gradient** ins Spiel:

**Definition:**
Der Gradient der Funktion $f \colon \mathbb{R}^{n} \to \mathbb{R}$ ist die Funktion

\begin{equation}
\nabla f \colon \mathbb{R}^{n} \to \mathbb{R}^{n};~~
x =
\begin{pmatrix}
x_{1} \\ \vdots \\ x_{n}
\end{pmatrix}
\mapsto
\begin{pmatrix}
\tfrac{\partial f}{\partial x_{1}} (x) \\ \vdots \\ \tfrac{\partial f}{\partial x_{n}} (x)
\end{pmatrix}.
\end{equation}

## **Anschauung:** Der Gradient zeigt immer in Richtung des stärksten Anstiegs.

Beispiel 1: $f(x, y) = x$, $\nabla f(x, y) = \begin{pmatrix} 1 \\ 0 \end{pmatrix}$.

In [ ]:
surface_plot = plot()
surface_plot.plot_surface(0,4,0,4,function = lambda x: x[0])
surface_plot.show()

Beispiel 2: $f(x, y) = \tfrac{1}{2} (x+y)$, $\nabla f(x, y) = \begin{pmatrix} \tfrac{1}{2} \\ \tfrac{1}{2} \end{pmatrix}$.

In [ ]:
surface_plot = plot()
surface_plot.plot_surface(0,4,0,4,function = lambda x: (1/3)*(x[0]+x[1]))
surface_plot.show()

## Zurück zur eigentlichen Funktion:

Der Gradient von
$𝑓(𝑥,𝑦)= \sin(𝑥)+0.05 𝑥^{2} + 0.1 𝑦^{2}$:

\begin{equation}
\nabla f(x, y)
=
\begin{pmatrix}
\frac{\partial f}{\partial x} \\ \frac{\partial f}{\partial y} \end{pmatrix}
=
\begin{pmatrix}
\cos(x) + 0.1x \\ 0.2 y
\end{pmatrix}
\end{equation}

Beispiele: 
\begin{equation}
\nabla f(-4, -2) =
\begin{pmatrix}
-1.05 \\ -0.40
\end{pmatrix},
\end{equation}
\begin{equation}
\nabla f(-1, 2) =
\begin{pmatrix}
0.44 \\ 0.20
\end{pmatrix}.
\end{equation}


In [ ]:
def grad_f(x):
    return np.array([np.cos(x[0])+0.1*x[0], 0.2*x[1]])

contour_plot.add_gradients(grad_f)
contour_plot.show()

## Die Schritte beim Gradientenverfahren

Wir starten mit einem Punkt $x_0 \in \mathbb{R}^{2}$.
Dann gehen wir im $n$-ten Schritt in die Gegenrichtung des Gradienten
$\nabla f(x, y)$, und zwar multipliziert mit der Lernrate $\gamma_n$:

\begin{equation}
	x_{n+1}= x_{n} - \gamma_{n} \nabla f({x} _{n}),\ n\geq 0.
\end{equation}

### Lernrate $\gamma = 1$:

Wir fangen im Punkt
$x_0 = \begin{pmatrix} -4 \\ -2 \end{pmatrix}$
an, und verwenden eine Lernrate von $\gamma = 1$.
Dann gilt

\begin{equation} \nabla f(x_0) =
\begin{pmatrix}\cos(-4)+0.1\cdot(-4) \\ 0.2\cdot(-2) \end{pmatrix}
=
\begin{pmatrix} -1.05 \\ -0.40 \end{pmatrix}
\end{equation}

und damit

\begin{equation}
x_{1} = x_{0} + \gamma_{0} \cdot \nabla f(x_0)
=
\begin{pmatrix} -4 \\ -2 \end{pmatrix}
- 1 \cdot 
\begin{pmatrix} -1.05 \\ -0.40 \end{pmatrix}
=
\begin{pmatrix}
-2.95 \\ -1.60
\end{pmatrix}.
\end{equation}
Die nächsten Schritte folgen analog:

$x_{2} = \begin{pmatrix} -1.67 \\ -1.28\end{pmatrix}, ~
x_{3} = \begin{pmatrix} -1.40 \\ -1.02\end{pmatrix}$

usw...

In [ ]:
# Das Gradientenverfahren mit einer Lernrate von gamma = 1:

def f(x):
    return np.sin(x[0]) + 0.05*x[0]**2 + 0.1*x[1]**2

def grad_f(x):
    return np.array([np.cos(x[0])+0.1*x[0], 0.2*x[1]])

contour_plot = plot()
contour_plot.plot_contour(-5,1,-3,3,f)
i = 1

In der folgenden Zelle werden die einzelnen Schritte des Gradientenverfahrens mit der Lernrate $\gamma = 1$ ausgeführt.
Wenn man diese Zelle mehrfach laufen lässt, sieht man, wie das Verfahren konvergiert.

In [ ]:
contour_plot.add_gradient_descent(x0=[-4, -2], function = f, grad=grad_f, gamma=1, Iterationen=i, color = "#636EFA")
contour_plot.show()
i += 1

### Das Gradientenverfahren mit einer Lernrate von $\gamma = 0.1$ ist langsamer:

In [ ]:
i = 1

In der folgenden Zelle werden die einzelnen Schritte des Gradientenverfahrens mit der Lernrate $\gamma = 0.1$ ausgeführt.
Beim mehrfachen Ausführen der Zelle sieht man, dass das Verfahren sehr langsam konvergiert.

In [ ]:
# Das Gradientenverfahren mit einer Lernrate von gamma = 0.1:

contour_plot.add_gradient_descent(x0=[-4, -2], function = f, grad=grad_f, gamma=0.1, Iterationen=i, color = "#EF553B")
contour_plot.show()
i+=1

### Eine Lernrate von $\gamma = 2$ führt hingegen nicht zu einer Konvergenz:

In [ ]:
i = 1

In der folgenden Zelle werden die einzelnen schritte der Gradientenverfahrens mit der Lernrate $\gamma = 2$ ausgeführt. 
Beim mehrfachen Ausführen der Zelle sieht man, dass das Verfahren nicht konvergiert, sondern die Lösung immer hin- und her springt.
Wenn es Probleme gibt, bitte noch mal die obere Zelle laufen lassen.

In [ ]:
# Das Gradientenverfahren mit einer Lernrate von gamma = 2:

contour_plot.add_gradient_descent(x0=[-4, -2], function = f, grad=grad_f, gamma=2, Iterationen=i, color = "#00CC96")
contour_plot.show()
i += 1

## Einfluss der Startwerts

Wenn die zu minimierende Funktion mehrere lokale Minima besitzt, spielt die Wahl des Startwertes auch eine große Rolle: 

In [ ]:
import numpy as np
def f(x):
    return np.sin(x[0]) + 0.05*x[0]**2 + 0.1*x[1]**2

contour_plot = plot()
contour_plot.plot_contour(-5,6,-3,3,f)

surface_plot = plot()
surface_plot.plot_surface(-5,6,-3,3,f)
surface_plot.show()

In [ ]:
import random.uniform

contour_plot.add_gradient_descent(x0=[random.uniform(-5,6),random.uniform(-3,3)], function = f, grad=grad_f, gamma=1, Iterationen=30)
contour_plot.show()

## Nichtlineare Optimierung mit Scipy

Das vorgestellte Verfahren ist für die Praxis noch zu einfach, in der praktischen Anwendung gibt es noch viele Probleme, die zu beachten sind.
Deswegen nimmt man normalerweise eine bereits vorhandene Implementierung.
Hierfür eignet sich das Python-Paket Scipy mit der Funktion `minimize` sehr gut.
Es reicht, die zu minimierende Funktion und den Startwert einzugeben:

In [ ]:
# Optimierung mit scipy
import numpy as np
from scipy.optimize import minimize

def f(x):
    return np.sin(x[0]) + 0.05*x[0]**2 + 0.1*x[1]**2

minimize(fun=f, x0=np.array([-4,-2]))

## Nebenbedingungen der Form $g(x) \leq 0$

Ein Beispiel für eine Nebenbedingung der Form $g(x) \leq 0$ ist
\begin{equation}
-(x+2)^{2} + y^{3} \leq 0
\end{equation}

Dies definiert einen Bereich, in dem sich die Lösung befinden kann.

In [ ]:
def h(x):
    return (x[0]+3)**3-x[1]

def g_(x):
    return - ((x+2)**2)**(1/3)

x1 = np.linspace(-5, -2, 100).tolist()
y1 = [np.sqrt(np.abs(2.25 - (x+3.5)**2)) for x in x1]
x2 = np.linspace(-2, -5, 100).tolist()
y2 = [-np.sqrt(np.abs(2.25 - (x+3.5)**2)) for x in x2]
x = x1 + x2
y = y1 + y2

contour_plot = plot()
contour_plot.plot_contour(-5,1,-3,3,f)

contour_plot.add_trace(go.Scatter(x=x, y=y, fill='tonexty', showlegend = False, marker = {'color' : '#19D3F3'})).show()

## Nebenbedingungen der Form $h(x) = 0$

Eine Nebenbedingung der Form $h(x) = 0$ definiert üblicherweise eine Linie, auf der sich die Lösung befindet.
Dies ist im Allgemeinen ein einschränkende Bedingung, d.h. es wird ein schlechteres Ergebnis als ohne Nebenbedingung erreicht.
Wir zeigen als Beispiel eine Nebenbedingung der Form
\begin{equation} h(x, y) = (x + 3)^2 - y. \end{equation}

In [ ]:
contour_plot.add_h()
contour_plot.show()

## Das quadratische Penalty-Verfahren

Wir wissen jetzt, wie man unrestringierte nichtlineare Optimierungsprobleme lösen kann.
Doch wie können wir die Nebenbedingungen berücksichtigen, also

\begin{equation}
\begin{split}
\text{Minimiere }& f(x) \\
\textbf{wobei }& \mathbf{g(x) \leq 0} \\
\textbf{und }& \mathbf{h(x) = 0} \textbf{ gilt?}
\end{split}
\end{equation}

Eine Möglichkeit ist es, zu der zu minimierenden Funktion $f$ einen Strafterm hinzuzuaddieren, der Abweichungen von den Nebenbedingungen bestraft.
Solche Verfahren heißen **Penalty-Verfahren**.
Wir stellen hier das quadratische **Penalty-Verfahren** vor.
Das Problem lässt sich dann als

\begin{equation}
\text{Minimiere } f(x) + \alpha \max(0, g(x))^{2} + \alpha h(x)^{2}, ~ \alpha > 0
\end{equation}

formulieren.

## Das Penalty-Verfahren für $h(x) = 0$:

Zur Erinnerung: 

\begin{equation}h(x, y) = (x + 3)^2 - y.\end{equation}

Wir wählen ein $\alpha > 0$ und müssen dann die Funktion

\begin{equation}
\begin{split}
f_{\text{pen}} (x, y)
&=
f(x, y) + \alpha h(x, y)^2
\end{split}
\end{equation}

minimieren.

In [ ]:
alpha = 0.01

def h(x):
    return (x[0]+3)**2 - x[1]

def f_pen(x):
    return f(x) + alpha*h(x)**2

surface_plot = plot()
surface_plot.plot_surface(-5,1,-3,3,f,opacity = 0.8, colorscale = 'blues')
surface_plot.plot_surface(-5,1,-3,3,f_pen, opacity = 1, showscale=False, colorscale = 'oranges')
surface_plot.show()
surface_plot.write_html("bla.html")

def grad_h_sq(x):
    return np.array([4*x[0]**3 + 36*x[0]**2 + 108*x[0] + 108 - 4*x[0]*x[1] - 12*x[1],
                     -2*x[0]**2 - 12*x[0] + 2*x[1] - 18])

contour_plot = plot()
contour_plot.plot_contour(-5,1,-3,3,f)
contour_plot.add_h()

contour_plot.result = [0, 3]

gamma = 1

x_range = 1
y_range = 1

Der Gradient von $h(x, y)^2$ ist

\begin{equation}
\nabla h^2(x,y) =
\begin{pmatrix}
4x^{3} + 36x^{2} + 108x + 108 - 4xy - 12y \\
-2x^{2} -12x + 2y - 18
\end{pmatrix}
\end{equation}

Der Gradient von $f_{ \text{pen}}$ ist

\begin{equation}
\nabla f_{ \text{pen}} = 
\nabla f + \alpha \nabla h.
\end{equation}

In [ ]:
contour_plot.add_gradient_descent(x0=contour_plot.result, function=f, grad=lambda x : (grad_f(x) + alpha*grad_h_sq(x)), gamma=gamma,
                                  Iterationen=100, Nebenbedingung = h)

contour_plot.show()

x_result = contour_plot.result[0]
y_result = contour_plot.result[1]

contour_plot.contour_zoom(x_result-x_range, x_result+x_range, y_result-y_range, y_result+y_range, f)

alpha *= 10
gamma /= 10

x_range /= 1.1
y_range /= 1.1

x0 = contour_plot.result

## Optimierung mit Nebenbedingungen in Python

Auch mit Nebenbedingungen kann man in Python gut optimieren.

### Optimierung mit $h(x, y) = 0$:

In [ ]:
# Optimierung mit scipy
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import NonlinearConstraint

def f(x):
    return np.sin(x[0]) + 0.05*x[0]**2 + 0.1*x[1]**2

def h(x):
    return (x[0]+3)**2 - x[1]

constraints = NonlinearConstraint(h, lb = 0, ub = 0)

minimize(fun=f, x0=np.array([3,0]), constraints=constraints)

### Optimierung mit $h(x, y) = 0$ und $g(x, y) \leq 0$:

In [ ]:
def g(x):
    return -(x[0]+2)**2 + x[1]**3

constraints = [NonlinearConstraint(h, lb = 0, ub = 0),
               NonlinearConstraint(g, lb = -np.inf, ub = 0)]

minimize(fun=f, x0=np.array([-4,-2]), constraints=constraints)

## Ausblick: Automatic Differentiation

### Numerische Berechnung bei nicht bekannten Gradienten:

\begin{equation}
\frac{\partial f}{\partial x} \approx \frac{f(x + h, y) - f(x, y)}{h}, ~ 
\frac{\partial f}{\partial y} \approx \frac{f(x, y + h) - f(x, y)}{h}, ~
h\text{ klein}
\end{equation}
 
$\Rightarrow$ Für jede partielle Ableitung muss ein eigener Differenzenquotient gebildet werden.

- Besonders anspruchsvoller Anwendungsfall nichtlinearer Optimierung: Neuronale Netze
- Gradientenverfahren wesentlicher Algorithmus zum Trainieren von neuronalen Netzen

### Anwendung bei neuronalen Netzen

![](simple_fnn_backprop.svg)
<figcaption align = "center"><b>Graphik zeigt ein neuronales Netz zusammen mit der Loss-Funktion. Quelle: Maren Hackenberg, AG Machine Learning, Institut für Medizinische Biometrie
und Statistik (IMBI), Universität Freiburg, Quelle: </b> <a href="https://github.com/maren-ha/Masterarbeit/blob/master/Figures/simple_fnn_backprop.pdf">github.com/maren-ha/Masterarbeit/blob/master/Figures/simple_fnn_backprop.pdf</a></figcaption>

- Loss-Funktion $\mathcal{L}$ hängt von sehr vielen Parametern ab (z.B. $10\,000 - 10\,000\,000$)
- numerische Berechnung des Gradienten aufwendig und rundungsfehleranfällig
- analytische Berechnung nicht praktikabel und abhängig von der genauen Struktur des Netzes

### Lösung: Automatic Differentiation

- Automatic differentiation-Pakete können von (vielen) in Python definierten Funktionen die exakte Ableitung berechnen
- Dadurch effizientere Optimierung hochdimensionaler Probleme möglich 
- dieses Optimierungsverfahren, Gradientenverfahren + automatic differentiation, ist zentrales Element für den Erfolg von vielen Machine Learning Methoden

### Beispiel: Effizienzsteigerung durch Automatic Differentiation

Wir minimieren die Funktion:
\begin{equation}
f(x) = \exp \left(- \sum_{i=1}^{n} i \cdot x_{i} \right) + \sum_{i=1}^{n} x_{i}^{2}
\end{equation}

In [ ]:
import autograd.numpy as np
from autograd import grad
from scipy.optimize import minimize
from time import time

def f(x):
    return np.exp(-np.sum(np.array(range(1, len(x)+1) * x))) + np.sum(x**2)

x0=np.ones(2)

## Ohne Gradient
minimize(fun=f, x0=x0)

In [ ]:
## Gradient berechnen
grad_f = grad(f)
minimize(fun=f, x0=x0, jac = grad_f)

In [ ]:
# Benchmarks

x0=np.ones(100)

t0 = time()
for i in range(10):
    fd = minimize(fun=f, x0=x0)
t_fd = (time() - t0) / 10

print("Funktionswert Finite Difference: ", fd.fun)
print("Rechenzeit Finite Difference: ", t_fd)

#hess_f = hessian(f)
t0 = time()
for i in range(10):
    grad_f = grad(f)
    autodiff = minimize(fun=f, x0=x0, jac = grad_f)
t_autodiff = (time() - t0) / 10

print("Funktionswert Autodiff: ", autodiff.fun)
print("Rechenzeit Autodiff: ", t_autodiff)
print("(Rechenzeit Finite Difference) / (Rechenzeit Autodiff):", t_fd / t_autodiff)